In [1]:
import pandas as pd
import os
import re
import sys
sys.path.append('/opt/homebrew/Caskroom/miniforge/base/lib/python3.10/site-packages')
import sklearn

import scipy



In [ ]:
scipy.interpolate.
RegularGridInterpolator

In [2]:
result_header_df = pd.read_excel('../MMV living lab data/data_headers.xlsx')

In [3]:
result_header_df

,Timestamp,FCU-1 Cooling Load_kW,FCU-2 Cooling Load_kW,FCU-3 Cooling Load_kW,FCU-4 Cooling Load_kW,FCU-5 Cooling Load_kW,PFCU-1 Cooling Load_kW,PFCU-2 Cooling Load_kW,Outdoor Temperature,Outdoor Temp Ave,...,Zone 1 CO2 - S2,Zone 2 Temperature,Zone 2 CO2,Zone 1 Ave CO2,Z7 Windows Open Close Status,Z6 Windows Open Close Status,Z5 Windows Open Close Status,Z1 Windows Open Close Status,Z2 Windows Open Close Status,Z3 Windows Open Close Status


In [4]:
index_df = pd.read_csv('../MMV living lab data/id_map.csv')

In [5]:
index_df

,id,name
0,1195,Barometric Pressure
1,1196,Dew Point Temperature
2,1279,FCU-01 CHW Flow Rate
3,1224,FCU-01 CHW Return Temp
4,1223,FCU-01 CHW Supply Temp
...,...,...
184,1083,Zone-03 Watt
185,1109,Zone-04 KWh
186,1084,Zone-04 Watt
187,1113,Zone-05 KWh


In [ ]:
index_df['name'].loc[index_df['id']==1092]

In [ ]:
index_df['id'].loc[index_df['name']=='FCU-01 CHW Return Temp']

In [ ]:
index_df['name']

In [11]:
#two directories:
#'../MMV living lab data/from BMS'
#'../MMV living lab data/from server'
directory  = '../MMV living lab data/from server'
df_dict = {}
try:
    for filename in os.listdir(directory):
        print(filename)
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            df_dict[filename] = pd.read_csv(filepath)
            #print(filepath)
            #print(df.shape)
            '''
            if not df.empty:
                try:
                    temp = pd.read_csv(filepath)
                    df = pd.concat([df,temp],ignore_index=True)
                except:
                    print('failed to read file: '+filename)
                    
            else:
                df = pd.read_csv(filepath)'''
                
except FileNotFoundError:
    print("Directory not found.")
except PermissionError:
    print("Permission denied.")


FCU1all.csv
FCU3all.csv
OAall.csv
FAN1all.csv
PFCU1all.csv
INall.csv
WINDOWSall.csv
FCU5all.csv
PLUGall.csv
FAN2all.csv
PFCU2all.csv
FCU2all.csv
.ipynb_checkpoints
FCU4all.csv


In [12]:
'''
    rules:
        if the current date == date+1 or date-1:
            if current date == date+1:
                keep the record
            else:
                if the difference between date-1 and date+1 is morethan or equal to 2:
                    set the current date time to be 1 minute ahead of current date
                else
                    drop the record as its a duplicated time step
                    From Data investigation the ones that are dropped are ones where both the 
                    previous and next date time are the same as the current timestamp
'''
'''
    rather than dropping duplicate values we could handle it by aggregating results that have the same time stamps
'''


for file in df_dict:
    df_dict[file]['date'] = pd.to_datetime(df_dict[file]['date'])
    
    df = df_dict[file]
    
    if len(df['date'].dt.second.unique())>1:
        print(file)
        #get index of all null values and drop them
        null_index = df_dict[file].loc[df_dict[file]["double_value"]=="\\N"].index
        df_dict[file] = df_dict[file].drop(null_index)
        
        #sort values and drop exact duplicates
        df_dict[file] = df_dict[file].sort_values(by = ['trending_id','date'])
        df_dict[file] = df_dict[file].drop_duplicates(keep='first')
        df_dict[file].reset_index(inplace=True,drop=True)
        
        #round date times to nearest minute and record previous and next timestamp
        df_dict[file]['date'] = df_dict[file]['date'].dt.round('min')
        df_dict[file]['date-1'] = df_dict[file].groupby(['trending_id'])['date'].shift(1)
        df_dict[file]['date+1'] = df_dict[file].groupby(['trending_id'])['date'].shift(-1)
        
        #get the index of all records where the previous timestamp is the same as the current
        #and the difference between the previous time stamp and the next time stamp is more than or equal to 2 seconds
        df_dict[file]['time_delta'] = (df_dict[file]['date+1']-df_dict[file]['date-1'])
        index_list =df_dict[file].loc[(df_dict[file]['date']==df_dict[file]['date-1'])&(df_dict[file]['time_delta']>=pd.Timedelta(2,'m'))].index

        #add one minute to the date time of the records found in the previous step
        df_dict[file]['date'].loc[index_list] = df_dict[file].loc[index_list]['date'] + pd.Timedelta(1,'m')
        
        #all of the remaining records where the current time stamp is the same as the previous can now be dropped
        #these records share the same time stamp as at least one other record
        #one other potential method would be to aggregate all records with shared values
        # this can be re-evaluated later
        drop_index = df_dict[file].loc[(df_dict[file]['date']==df_dict[file]['date-1'])].index
        df_dict[file] = df_dict[file].drop(drop_index)
        
    elif df['date'].dt.second.unique()[0] != 0 :
        #here we can assert that the set of potential seconds has a length of 1
        #the only problem is that the value is not 0 and so all we need to do is
        #round all values to nearest minute as they will all be rounded in the same way
        df_dict[file]['date'] = df_dict[file]['date'].dt.round('min')
    

OAall.csv


/var/folders/t5/__569tsd0s9c8lms_ltt1k000000gn/T/ipykernel_84218/3798696051.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict[file]['date'].loc[index_list] = df_dict[file].loc[index_list]['date'] + pd.Timedelta(1,'m')


WINDOWSall.csv


/var/folders/t5/__569tsd0s9c8lms_ltt1k000000gn/T/ipykernel_84218/3798696051.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict[file]['date'].loc[index_list] = df_dict[file].loc[index_list]['date'] + pd.Timedelta(1,'m')


In [13]:
comp_df = pd.DataFrame()
for file in df_dict:
    #iterate through each data frame
    df = df_dict[file]
    
    for trending_id in df.trending_id.unique():
        #for each trending ID we will create a subset data frame
        #each subset dataframe will be combined to create a wide data set 
        #rather than a long dataset
        temp = df.loc[df['trending_id']==trending_id]
        
        #the names are taken from the index dataframe
        temp = temp.rename(columns = {'double_value':index_df['name'].loc[index_df['id']==trending_id].values[0]})
        
        #we just check that the data frame that we are merging into isnt empty and if it is we handle it a bit differently
        if not comp_df.empty:
            comp_df = comp_df.merge(temp[['date',index_df['name'].loc[index_df['id']==trending_id].values[0]]], how='outer', on= 'date')
        else:
            comp_df = temp[['date',index_df['name'].loc[index_df['id']==trending_id].values[0]]]


In [14]:
comp_df

,date,FCU-01 Watt,FCU-1 Cooling Load_kW,FCU-01 KWh,FCU-01 CHW Supply Temp,FCU-01 CHW Return Temp,FCU-01 Return Air CO2 Level,FCU-01 Return Air Humi,FCU-01 Return Temp,FCU-01 Supply Air Flow Rate,...,FCU-04 Watt,FCU-4 Cooling Load_kW,FCU-04 CHW Return Temp,FCU-04 CHW Supply Temp,FCU-04 Return Air CO2 Level,FCU-04 Return Air Humi,FCU-04 RM Temp,FCU-04 Supply Air Flow Rate,FCU-04 Supply Air Humi,FCU-04 CHW Flow Rate
0,2024-03-13 08:39:00,238.524742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,242.659332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-03-13 08:40:00,237.809601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241.452957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-03-13 08:41:00,237.418716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241.703972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-03-13 08:42:00,237.418716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,240.435181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-03-13 08:43:00,236.533173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,240.800247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284963,2024-02-02 05:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284964,2024-02-02 05:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284965,2024-02-02 05:06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284966,2024-02-02 05:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
comp_df.to_csv('ALL_SERVER.csv')

In [ ]:
comp_df.date.max()-comp_df.date.min()

In [ ]:
comp_df=pd.read_csv('test.csv')

In [ ]:
comp_df = comp_df.sort_values(by=['date'])
comp_df.reset_index(inplace=True,drop=False)

In [ ]:
comp_df['date'] = pd.to_datetime(comp_df['date'])

In [ ]:
comp_df['date'] = comp_df['date'].dt.round('min')
comp_df['date-1'] = comp_df['date'].shift(1)
comp_df['date+1'] = comp_df['date'].shift(-1)
        

In [ ]:
comp_df['delta'] = comp_df['date']-comp_df['date-1']

In [ ]:
comp_df.loc[comp_df['delta']>pd.Timedelta(1,'m')]['delta'].std()

In [ ]:
comp_df.loc[comp_df['delta']>pd.Timedelta(1,'m')]

In [ ]:
'''
notes on what needs to be done:

the dataset has ocasional gaps (gaps more than the sample rate of 1 min) varying in size
    average: 1 days 21 hours 57 minutes 16.363636363 seconds
    mode: 2 minutes
    median 5 minutes 30 seconds
    std: 6 days 21 hours 41 minutes 25.867397707 seconds
    
the mean of the gaps is dominated by extremely large gaps
I propose that for data gaps < 10 mins we generate Null records

if the gap between two known measurements is less than or equal to 10 mins we do a linear interpolation between the two measurements
if the gap is larger then we do nothing

these can all be changes later. The types of interpolation can also be chosen based off of what other columns we have available aswell as
other factors

I need to Transform the columns into the desired output format
'''
    

In [ ]:
len(comp_df.columns)

In [ ]:
len(result_header_df.columns)

In [ ]:
for col in result_header_df.columns:
    if not (col in comp_df.columns):
        print(col)

In [ ]:
comp_df['Timestamp'] = comp_df['date']

In [ ]:
comp_df = comp_df[result_header_df.columns]

In [ ]:
comp_df.to_csv('Raw_data_from_BMS.csv')

In [16]:
server = pd.read_csv('ALL_SERVER.csv')
bms = pd.read_csv('ALL_BMS.csv')

In [29]:
merged = pd.merge(server, bms, on=list(server.columns), how="outer")

In [30]:
merged

,Unnamed: 0,date,FCU-01 Watt,FCU-1 Cooling Load_kW,FCU-01 KWh,FCU-01 CHW Supply Temp,FCU-01 CHW Return Temp,FCU-01 Return Air CO2 Level,FCU-01 Return Air Humi,FCU-01 Return Temp,...,FCU-04 Watt,FCU-4 Cooling Load_kW,FCU-04 CHW Return Temp,FCU-04 CHW Supply Temp,FCU-04 Return Air CO2 Level,FCU-04 Return Air Humi,FCU-04 RM Temp,FCU-04 Supply Air Flow Rate,FCU-04 Supply Air Humi,FCU-04 CHW Flow Rate
0,0,2024-03-13 08:39:00,238.524742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,242.659332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2024-03-13 08:40:00,237.809601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241.452957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2024-03-13 08:41:00,237.418716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,241.703972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2024-03-13 08:42:00,237.418716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,240.435181,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2024-03-13 08:43:00,236.533173,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,240.800247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284963,284963,2024-02-02 05:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284964,284964,2024-02-02 05:04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284965,284965,2024-02-02 05:06:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284966,284966,2024-02-02 05:07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
duplicates = merged['date'].duplicated()

In [33]:
duplicates.sum()
#no duplicated timestamps

0

In [35]:
merged.to_csv('data_from_server_bms.csv')